# MLP 실습 : ElasticNet

출처 : http://www.databaser.net/moniwiki/wiki.php/Python-LinearRegression

<img src="http://www.databaser.net/moniwiki/pds/Python_2dLinearRegression/lm_compare.png">

## data/library 불러오기

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
data=pd.read_csv("house_price.csv")
x=data[data.columns[1:5]]
y=data[['house_value']]

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
x_train,x_test,y_train,y_test= train_test_split(x,y,random_state=42)
scaler.fit(x_train)
x_scaled_train=scaler.transform(x_train)
x_scaled_test=scaler.transform(x_test)

In [15]:
from sklearn.linear_model import ElasticNet
model=ElasticNet()

In [16]:
dir(model)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_n_features',
 '_decision_function',
 '_estimator_type',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_preprocess_data',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_set_intercept',
 '_validate_data',
 'alpha',
 'copy_X',
 'fit',
 'fit_intercept',
 'get_params',
 'l1_ratio',
 'max_iter',
 'normalize',
 'path',
 'positive',
 'precompute',
 'predict',
 'random_state',
 'score',
 'selection',
 'set_params',
 'sparse_coef_',
 'tol',
 'warm_start']

## model 학습하기

In [17]:
model.fit(x_scaled_train,y_train)

ElasticNet()

In [18]:
pred_train=model.predict(x_scaled_train)
pred_test=model.predict(x_scaled_test)

In [19]:
model.score(x_scaled_train,y_train)

0.050029698219161034

In [20]:
model.score(x_scaled_test,y_test)

0.051683303919568435

## RMSE 확인하기

In [23]:
import numpy as np
from sklearn.metrics import mean_squared_error
MSE_train=mean_squared_error(y_train,pred_train)
MSE_test=mean_squared_error(y_test,pred_test)
print(MSE_train)
print(MSE_test)

8653904860.833986
8667190548.019886


## Hyper Parameter 최적화

### GridSearch

In [24]:
param_grid={'alpha':[0.0,1e-6,1e-5,1e-4,1e-3,1e-2,0.1,0.5,1,2.0,3.0]}

In [25]:
from sklearn.model_selection import GridSearchCV

In [27]:
grid_search=GridSearchCV(ElasticNet(),param_grid,cv=5)
grid_search.fit(x_scaled_train,y_train)

GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [0.0, 1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1,
                                   0.5, 1, 2.0, 3.0]})

In [28]:
print("Best Parameter : {}".format(grid_search.best_params_))
print("Best Cross-validity Score : {:.4f}".format(grid_search.best_score_))
print('Test set Score : {:.4f}'.format(grid_search.score(x_scaled_test,y_test)))

Best Parameter : {'alpha': 1e-05}
Best Cross-validity Score : 0.5452
Test set Score : 0.5627


### RandomSearch

In [29]:
from scipy.stats import randint

In [30]:
param_distribs={"alpha":randint(low=0.00001,high=10)}

In [34]:
from sklearn.model_selection import RandomizedSearchCV
random_search=RandomizedSearchCV(ElasticNet(),param_distributions=param_distribs,n_iter=100,cv=5)
random_search.fit(x_scaled_train,y_train)

RandomizedSearchCV(cv=5, estimator=ElasticNet(), n_iter=100,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001AEDC6F0520>})

In [35]:
print("Best Parameter : {}".format(random_search.best_params_))
print("Best Cross-validity Score : {:.4f}".format(random_search.best_score_))
print('Test set Score : {:.4f}'.format(random_search.score(x_scaled_test,y_test)))

Best Parameter : {'alpha': 0}
Best Cross-validity Score : 0.5452
Test set Score : 0.5627
